In [ ]:
from typing import Tuple

import numpy as np
from scipy.spatial.transform import Rotation as R
import pandas as pd

import mujoco
from mujoco import viewer

from dm_control import composer, mjcf
from dm_robotics.moma.models.arenas import empty
from dm_robotics.moma import robot

from ikpy.chain import Chain

from hydra import compose, initialize
from hydra.utils import instantiate


# load different robot configurations
initialize(version_base=None, config_path="./config", job_name="default_config")
POSITION_CONFIG = compose(config_name="controller_tuning", overrides=["robots=default"])
VELOCITY_CONFIG = compose(config_name="controller_tuning", overrides=["robots=velocity"])
MOTOR_CONFIG = compose(config_name="controller_tuning", overrides=["robots=motor"])
IKPY_URDF_PATH = "./models/arms/robot.urdf"

# For now assign default cfg
cfg = MOTOR_CONFIG
kinematic_chain = Chain.from_urdf_file(IKPY_URDF_PATH, base_elements=["panda_link0"]) 

# Building Controllers on Torque Actuator

Example of running a controller that interacts with the motor actuator model.

## Load Scene

In [ ]:
def build_arena(name: str) -> composer.Arena:
    """Build a MuJoCo arena."""
    arena = empty.Arena(name=name)
    arena.mjcf_model.option.timestep = 0.001
    arena.mjcf_model.option.gravity = (0.0, 0.0, -9.8)
    arena.mjcf_model.size.nconmax = 1000
    arena.mjcf_model.size.njmax = 2000
    arena.mjcf_model.visual.__getattr__("global").offheight = 640
    arena.mjcf_model.visual.__getattr__("global").offwidth = 640
    arena.mjcf_model.visual.map.znear = 0.0005
    return arena


def add_robot_and_gripper(arena: composer.Arena, arm, gripper) -> Tuple[composer.Entity, composer.Entity]:
    """Add a robot and gripper to the arena."""
    # attach the gripper to the robot
    robot.standard_compose(arm=arm, gripper=gripper)

    # define robot base site
    robot_base_site = arena.mjcf_model.worldbody.add(
        "site",
        name="robot_base",
        pos=(0.0, 0.0, 0.0),
    )

    # add the robot and gripper to the arena
    arena.attach(arm, robot_base_site)

    return arm, gripper

# build the base arena
arena = build_arena("base_scene")

# add robot arm and gripper to the arena
arm = instantiate(cfg.robots.arm)
gripper = instantiate(cfg.robots.gripper)
arm, gripper = add_robot_and_gripper(arena, arm, gripper)

physics = mjcf.Physics.from_mjcf_model(arena.mjcf_model)

# set the default arm joint positions to ready
physics.data.qpos[:7] = np.array([0, -0.785, 0, -2.356, 0, 1.571, 0.785])

# launch passive viewer
passive_view = viewer.launch_passive(physics.model._model, physics.data._data)

## Try Stepping Simulation

In [ ]:
physics.reset()
physics.data.qpos[:7] = np.array([0, -0.785, 0, -2.356, 0, 1.571, 0.785])
while True:
    physics.set_control(np.ones(8)*80)
    physics.step()
    passive_view.sync()

## Implement and Test Controller

In [ ]:
physics.reset()
physics.data.qpos[:7] = np.array([0, -0.785, 0, -2.356, 0, 1.571, 0.785])

# come up with targets
target_eef_pose = kinematic_chain.forward_kinematics(
                joints = np.array([0.0, -2.0, -0.785, 0.0, -2.356, 0.0, 1.571, 0.785, 0.0]),
                full_kinematics = False,
                )[:3, 3]
target_orientation = R.from_euler('xyz', [0, 180, 0], degrees=True).as_matrix()

# convert to joint target
joint_target = kinematic_chain.inverse_kinematics(
            target_position = target_eef_pose,
            target_orientation = target_orientation,
            orientation_mode = "all",
            initial_position = np.array([0.0, 0.0, -0.785, 0.0, -2.356, 0.0, 1.571, 0.785, 0.0]),
            )[1:-1]

# PD control with inverse dynamics
kp = 200
kd = 100
duration = 5.0

joint_pos_data = pd.DataFrame(columns=["joint_1", "joint_2", "joint_3", "joint_4", "joint_5", "joint_6", "joint_7"])
joint_vel_data = pd.DataFrame(columns=["joint_1", "joint_2", "joint_3", "joint_4", "joint_5", "joint_6", "joint_7"])

while physics.data.time < duration:
    current_joint_position = physics.data.qpos[:7]
    current_joint_velocity = physics.data.qvel[:7]
    joint_pos_data = pd.concat([joint_pos_data, pd.DataFrame([current_joint_position], columns=joint_pos_data.columns)], ignore_index=True) 
    joint_vel_data = pd.concat([joint_vel_data, pd.DataFrame([current_joint_velocity], columns=joint_vel_data.columns)], ignore_index=True)
    
    # calculate acceleration
    target_acc = kp * (joint_target - current_joint_position) - kd * current_joint_velocity
    prev = current_joint_velocity.copy()

    # calculate torque
    physics.data.qacc[:7] = target_acc
    mujoco.mj_inverse(physics.model._model, physics.data._data)
    sol = physics.data.qfrc_inverse[:7].copy()
    physics.data.qacc[:7] = prev

    # apply torque and update physics
    sol = np.concatenate([sol, [255.0]])
    physics.set_control(sol)
    physics.step()
    passive_view.sync()


# plot control curves
# plot joint angles and velocities
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig_pos = make_subplots(rows=3, cols=3, vertical_spacing=0.02)
fig_pos.add_trace(go.Scatter(y=joint_pos_data["joint_1"], name="joint_1"), row=1, col=1)
fig_pos.add_trace(go.Scatter(y=joint_pos_data["joint_2"], name="joint_2"), row=1, col=2)
fig_pos.add_trace(go.Scatter(y=joint_pos_data["joint_3"], name="joint_3"), row=1, col=3)
fig_pos.add_trace(go.Scatter(y=joint_pos_data["joint_4"], name="joint_4"), row=2, col=1)
fig_pos.add_trace(go.Scatter(y=joint_pos_data["joint_5"], name="joint_5"), row=2, col=2)
fig_pos.add_trace(go.Scatter(y=joint_pos_data["joint_6"], name="joint_6"), row=2, col=3)
fig_pos.add_trace(go.Scatter(y=joint_pos_data["joint_7"], name="joint_7"), row=3, col=1)
fig_pos.update_layout(
        height=600, 
        width=800, 
        title_text="Joint Angles",
        yaxis_title="Joint Angle (rad)",
        )

# add constant reference line
fig_pos.add_hline(y=joint_target[0], line_dash="dash", row=1, col=1)
fig_pos.add_hline(y=joint_target[1], line_dash="dash", row=1, col=2)
fig_pos.add_hline(y=joint_target[2], line_dash="dash", row=1, col=3)
fig_pos.add_hline(y=joint_target[3], line_dash="dash", row=2, col=1)
fig_pos.add_hline(y=joint_target[4], line_dash="dash", row=2, col=2)
fig_pos.add_hline(y=joint_target[5], line_dash="dash", row=2, col=3)
fig_pos.add_hline(y=joint_target[6], line_dash="dash", row=3, col=1)


fig_vel = make_subplots(rows=3, cols=3, vertical_spacing=0.02)
fig_vel.add_trace(go.Scatter(y=joint_vel_data["joint_1"], name="joint_1"), row=1, col=1)
fig_vel.add_trace(go.Scatter(y=joint_vel_data["joint_2"], name="joint_2"), row=1, col=2)
fig_vel.add_trace(go.Scatter(y=joint_vel_data["joint_3"], name="joint_3"), row=1, col=3)
fig_vel.add_trace(go.Scatter(y=joint_vel_data["joint_4"], name="joint_4"), row=2, col=1)
fig_vel.add_trace(go.Scatter(y=joint_vel_data["joint_5"], name="joint_5"), row=2, col=2)
fig_vel.add_trace(go.Scatter(y=joint_vel_data["joint_6"], name="joint_6"), row=2, col=3)
fig_vel.add_trace(go.Scatter(y=joint_vel_data["joint_7"], name="joint_7"), row=3, col=1)
fig_vel.update_layout(
        height=600, 
        width=800, 
        title_text="Joint Velocities",
        yaxis_title="Joint Velocity (rad/s)",
        )

fig_pos.show()
fig_vel.show()

# Automate Tuning of Actuator Params

## Position Actuator

## Velocity Actuator